In [2]:
# Імпорт необхідних бібліотек
import pandas as pd
import numpy as np
import timeit
import io
import requests

# Налаштування відображення для зручності
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [3]:
# Шлях до файлу. Змініть, якщо файл знаходиться в іншій директорії.
file_path = 'household_power_consumption.txt'

try:
    # Читання датасету
    # Використовуємо sep=';' та '?' як значення, що відсутні (na_values)
    print(f"Початок читання файлу: {file_path}")
    df = pd.read_csv(
        file_path,
        sep=';',
        parse_dates={'DateTime': ['Date', 'Time']},
        infer_datetime_format=True,
        na_values=['?'],
        low_memory=False
    )
    print("Датасет успішно завантажено.")
    print(f"Форма датасету: {df.shape}")
except FileNotFoundError:
    print(f"Помилка: Файл '{file_path}' не знайдено.")
    print("Будь ласка, завантажте 'household_power_consumption.txt' у поточну директорію або змініть 'file_path'.")
    df = pd.DataFrame() # Створюємо порожній DataFrame для запобігання помилок

Початок читання файлу: household_power_consumption.txt


C:\Users\Alexander\AppData\Local\Temp\ipykernel_23972\907263137.py:8: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
C:\Users\Alexander\AppData\Local\Temp\ipykernel_23972\907263137.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(


Датасет успішно завантажено.
Форма датасету: (2075259, 8)


C:\Users\Alexander\AppData\Local\Temp\ipykernel_23972\907263137.py:8: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(


In [4]:
if 'df' in locals() and not df.empty:
    print("Початок Data Cleaning...")
    
    # Виводимо кількість пропущених значень до очищення
    print("\nКількість NaN до очищення:")
    print(df.isnull().sum())
    
    # Завдання вимагає обробити пропущені дані.
    # Оскільки це часовий ряд і велика кількість даних, а пропущені значення
    # складають лише 1.25% від загальної кількості, найпоширенішими та
    # ефективними методами є:
    # 1. Видалення рядків з NaN (якщо їх мало).
    # 2. Заміна середнім/медіаною/попереднім значенням.
    
    # Використаємо простий метод: заповнення NaN медіаною для числових колонок.
    # Це допомагає зберегти розмір вибірки і менш чутливе до викидів, ніж середнє.
    
    # Конвертуємо всі колонки, крім першої ('DateTime'), у float,
    # оскільки read_csv міг інтерпретувати їх як object через наявність '?'
    cols_to_convert = df.columns.drop('DateTime')
    for col in cols_to_convert:
        # Використовуємо .loc для уникнення SettingWithCopyWarning
        df.loc[:, col] = pd.to_numeric(df.loc[:, col], errors='coerce')
        
    # Заповнення NaN медіаною
    median_values = df.median(numeric_only=True)
    df_cleaned = df.fillna(median_values)
    
    # Виводимо кількість пропущених значень після очищення
    print("\nКількість NaN після очищення:")
    print(df_cleaned.isnull().sum())
    
    # Встановлюємо 'DateTime' як індекс для зручності роботи з часовими рядами
    df_cleaned.set_index('DateTime', inplace=True)
    
    print("Data Cleaning завершено. Датафрейм готовий до аналізу.")
else:
    print("Неможливо виконати Data Cleaning, оскільки датафрейм порожній або не завантажений.")
    df_cleaned = pd.DataFrame()

Початок Data Cleaning...

Кількість NaN до очищення:
DateTime                     0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

Кількість NaN після очищення:
DateTime                 0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64
Data Cleaning завершено. Датафрейм готовий до аналізу.


In [5]:
def select_high_active_power(data, threshold=5):
    """Обирає записи, де Global_active_power перевищує поріг (кВт)."""
    return data[data['Global_active_power'] > threshold]

def select_current_and_submetering(data, current_min=19, current_max=20):
    """
    Обирає записи за силою струму (19-20 А) та умовою:
    Sub_metering_2 (Пральна машина, Холодильник) > Sub_metering_3 (Бойлер, Кондиціонер).
    """
    # Фільтрація за силою струму
    current_filter = (data['Global_intensity'] >= current_min) & \
                     (data['Global_intensity'] <= current_max)
    
    df_filtered_current = data[current_filter].copy()
    
    # Фільтрація за споживанням (група 2 > група 3)
    consumption_filter = df_filtered_current['Sub_metering_2'] > df_filtered_current['Sub_metering_3']
    
    return df_filtered_current[consumption_filter]

if not df_cleaned.empty:
    # 1. Вибірка: Загальна активна потужність > 5 кВт
    time_high_power = timeit.timeit(
        lambda: select_high_active_power(df_cleaned, 5),
        number=1
    )
    df_high_power = select_high_active_power(df_cleaned, 5)
    print(f"1. Час виконання (Потужність > 5 кВт): {time_high_power:.6f} сек.")
    print(f"   Розмір вибірки 1: {df_high_power.shape}")
    
    # 2. Вибірка: 19 A <= Сила струму <= 20 A та Sub_metering_2 > Sub_metering_3
    time_current_submetering = timeit.timeit(
        lambda: select_current_and_submetering(df_cleaned),
        number=1
    )
    df_current_submetering = select_current_and_submetering(df_cleaned)
    print(f"2. Час виконання (Струм 19-20 А, S2 > S3): {time_current_submetering:.6f} сек.")
    print(f"   Розмір вибірки 2: {df_current_submetering.shape}")

1. Час виконання (Потужність > 5 кВт): 0.008501 сек.
   Розмір вибірки 1: (17547, 7)
2. Час виконання (Струм 19-20 А, S2 > S3): 0.008257 сек.
   Розмір вибірки 2: (2509, 7)


In [6]:
def select_random_sample_and_calculate_mean(data, n_samples=500000):
    """
    Обирає n випадкових записів без повторів та обчислює середні 
    для Sub_metering 1, 2, 3.
    """
    # Випадкова вибірка без повторів
    df_sample = data.sample(n=n_samples, replace=False, random_state=42)
    
    # Обчислення середніх
    mean_sub1 = df_sample['Sub_metering_1'].mean()
    mean_sub2 = df_sample['Sub_metering_2'].mean()
    mean_sub3 = df_sample['Sub_metering_3'].mean()
    
    return df_sample, (mean_sub1, mean_sub2, mean_sub3)

def select_complex_time_based(data, power_threshold=6):
    """
    Обирає записи після 18:00 з потужністю > 6 кВт, 
    де Sub_metering_2 є найбільшою,
    а потім застосовує складну індексацію.
    """
    # 1. Фільтрація за часом (після 18-00) та потужністю (> 6 кВт)
    # Зверніть увагу: індекс вже є DateTimeIndex
    time_power_filter = (data.index.hour >= 18) & \
                        (data['Global_active_power'] > power_threshold)
    df_filtered_time = data[time_power_filter].copy()
    
    # 2. Визначення, де Sub_metering_2 є найбільшою
    # Використовуємо .values для ефективнішого порівняння
    s2_max_filter = (df_filtered_time['Sub_metering_2'].values > df_filtered_time['Sub_metering_1'].values) & \
                    (df_filtered_time['Sub_metering_2'].values > df_filtered_time['Sub_metering_3'].values)
    df_s2_max = df_filtered_time[s2_max_filter]
    
    # 3. Складна індексація
    n_rows = len(df_s2_max)
    half_point = n_rows // 2
    
    # Перша половина: кожен третій результат
    first_half = df_s2_max.iloc[:half_point:3]
    
    # Друга половина: кожен четвертий результат
    second_half = df_s2_max.iloc[half_point::4]
    
    # Об'єднання результатів
    final_selection = pd.concat([first_half, second_half])
    
    return final_selection

if not df_cleaned.empty:
    # 3. Вибірка: Випадкові 500000 записів та обчислення середніх
    time_random_sample = timeit.timeit(
        lambda: select_random_sample_and_calculate_mean(df_cleaned),
        number=1
    )
    df_random_sample, means = select_random_sample_and_calculate_mean(df_cleaned)
    print(f"3. Час виконання (Випадкова вибірка 500k): {time_random_sample:.6f} сек.")
    print(f"   Розмір вибірки 3: {df_random_sample.shape}")
    print(f"   Середні Sub_metering: S1={means[0]:.3f}, S2={means[1]:.3f}, S3={means[2]:.3f}")
    
    # 4. Вибірка: Складна часова фільтрація та індексація
    time_complex_selection = timeit.timeit(
        lambda: select_complex_time_based(df_cleaned),
        number=1
    )
    df_complex_selection = select_complex_time_based(df_cleaned)
    print(f"4. Час виконання (Складна вибірка по часу): {time_complex_selection:.6f} сек.")
    print(f"   Розмір вибірки 4: {df_complex_selection.shape}")
    
    # Вибираємо випадкову вибірку для подальших кроків (нормування, кореляція, OHE)
    df_for_analysis = df_random_sample.copy()
else:
    print("Пропуск операцій, оскільки датафрейм порожній.")

3. Час виконання (Випадкова вибірка 500k): 0.108933 сек.
   Розмір вибірки 3: (500000, 7)
   Середні Sub_metering: S1=1.109, S2=1.284, S3=6.397
4. Час виконання (Складна вибірка по часу): 0.044558 сек.
   Розмір вибірки 4: (310, 7)


In [7]:
if not df_for_analysis.empty:
    # Вибираємо числові колонки для нормування/стандартизації
    numeric_cols = df_for_analysis.columns.tolist()
    
    # Створюємо копію для операцій
    df_norm_std = df_for_analysis.copy()

    # --- 1. Нормування (Min-Max Scaling) ---
    def min_max_normalize(series):
        return (series - series.min()) / (series.max() - series.min())

    time_normalize = timeit.timeit(
        lambda: df_norm_std[numeric_cols].apply(min_max_normalize, axis=0),
        number=1
    )
    df_normalized = df_norm_std[numeric_cols].apply(min_max_normalize, axis=0)
    df_normalized.columns = [col + '_Normalized' for col in numeric_cols]
    
    print(f"Час виконання (Нормування Min-Max): {time_normalize:.6f} сек.")
    
    # --- 2. Стандартизація (Z-score Scaling) ---
    def z_score_standardize(series):
        return (series - series.mean()) / series.std()

    time_standardize = timeit.timeit(
        lambda: df_norm_std[numeric_cols].apply(z_score_standardize, axis=0),
        number=1
    )
    df_standardized = df_norm_std[numeric_cols].apply(z_score_standardize, axis=0)
    df_standardized.columns = [col + '_Standardized' for col in numeric_cols]
    
    print(f"Час виконання (Стандартизація Z-score): {time_standardize:.6f} сек.")
    
    # Об'єднання в один DataFrame для перевірки
    df_processed = pd.concat([df_norm_std, df_normalized, df_standardized], axis=1)
    
    print("\nПерші 5 рядків нормованого/стандартизованого датасету:")
    print(df_processed[['Global_active_power', 'Global_active_power_Normalized', 'Global_active_power_Standardized']].head())
else:
    print("Пропуск нормування та стандартизації, оскільки датафрейм для аналізу порожній.")

Час виконання (Нормування Min-Max): 0.042353 сек.
Час виконання (Стандартизація Z-score): 0.067945 сек.

Перші 5 рядків нормованого/стандартизованого датасету:
                     Global_active_power  Global_active_power_Normalized  Global_active_power_Standardized
DateTime                                                                                                  
2010-07-07 18:10:00                0.256                        0.016837                         -0.789222
2007-05-14 06:50:00                0.466                        0.036701                         -0.589539
2007-09-26 18:10:00                0.758                        0.064321                         -0.311885
2007-06-19 07:30:00                1.290                        0.114642                          0.193979
2010-05-10 04:43:00                0.428                        0.033106                         -0.625672


In [8]:
if not df_for_analysis.empty:
    attr1 = 'Global_active_power'
    attr2 = 'Global_reactive_power'

    print(f"Обчислення кореляції між '{attr1}' та '{attr2}':")

    # --- 1. Коефіцієнт Пірсона (Pearson) ---
    time_pearson = timeit.timeit(
        lambda: df_for_analysis[[attr1, attr2]].corr(method='pearson').iloc[0, 1],
        number=1
    )
    pearson_corr = df_for_analysis[[attr1, attr2]].corr(method='pearson').iloc[0, 1]
    
    print(f"   Коефіцієнт Пірсона: {pearson_corr:.6f} (Час: {time_pearson:.6f} сек.)")

    # --- 2. Коефіцієнт Спірмена (Spearman) ---
    time_spearman = timeit.timeit(
        lambda: df_for_analysis[[attr1, attr2]].corr(method='spearman').iloc[0, 1],
        number=1
    )
    spearman_corr = df_for_analysis[[attr1, attr2]].corr(method='spearman').iloc[0, 1]
    
    print(f"   Коефіцієнт Спірмена: {spearman_corr:.6f} (Час: {time_spearman:.6f} сек.)")
else:
    print("Пропуск обчислення кореляцій, оскільки датафрейм для аналізу порожній.")

Обчислення кореляції між 'Global_active_power' та 'Global_reactive_power':
   Коефіцієнт Пірсона: 0.247746 (Час: 0.009096 сек.)
   Коефіцієнт Спірмена: 0.265113 (Час: 0.089634 сек.)


In [10]:
if not df_for_analysis.empty:
    df_ohe = df_for_analysis.copy()
    
    # --- 1. Створення категоріального атрибута (Година доби) ---
    def get_time_of_day(hour):
        if 5 <= hour < 12:
            return 'Morning'
        elif 12 <= hour < 18:
            return 'Day'
        elif 18 <= hour < 22:
            return 'Evening'
        else:
            return 'Night'

    # Створюємо нову колонку 'Time_of_Day'
    df_ohe['Time_of_Day'] = df_ohe.index.hour.map(get_time_of_day)
    
    print("Створено категоріальний атрибут 'Time_of_Day'.")
    print(df_ohe['Time_of_Day'].value_counts())
    
    # --- 2. Виконання One Hot Encoding (OHE) ---
    category_col = 'Time_of_Day'
    
    # Використовуємо pd.get_dummies, яка за замовчуванням видаляє оригінальну колонку
    time_ohe = timeit.timeit(
        lambda: pd.get_dummies(df_ohe, columns=[category_col], prefix=category_col),
        number=1
    )
    df_ohe_encoded = pd.get_dummies(df_ohe, columns=[category_col], prefix=category_col)
    
    print(f"\nЧас виконання (One Hot Encoding): {time_ohe:.6f} сек.")
    
    print("\nПерші 5 рядків датафрейму після OHE:")
    # Виводимо тільки НОВІ, закодовані колонки
    # (оригінальна колонка 'Time_of_Day' була видалена, тому її не можна вивести)
    ohe_cols = [col for col in df_ohe_encoded.columns if col.startswith(f'{category_col}_')]
    
    # Виводимо частину оригінального датафрейму (для контексту) та нові колонки OHE
    cols_to_show = ['Global_active_power'] + ohe_cols
    print(df_ohe_encoded[cols_to_show].head())
    
    print(f"\nКількість колонок після OHE: {df_ohe_encoded.shape[1]}")
else:
    print("Пропуск One Hot Encoding, оскільки датафрейм для аналізу порожній.")

Створено категоріальний атрибут 'Time_of_Day'.
Time_of_Day
Morning    145810
Night      145751
Day        125050
Evening     83389
Name: count, dtype: int64

Час виконання (One Hot Encoding): 0.046492 сек.

Перші 5 рядків датафрейму після OHE:
                     Global_active_power  Time_of_Day_Day  Time_of_Day_Evening  Time_of_Day_Morning  Time_of_Day_Night
DateTime                                                                                                              
2010-07-07 18:10:00                0.256            False                 True                False              False
2007-05-14 06:50:00                0.466            False                False                 True              False
2007-09-26 18:10:00                0.758            False                 True                False              False
2007-06-19 07:30:00                1.290            False                False                 True              False
2010-05-10 04:43:00                0.428  